In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import matplotlib.pyplot as plt

D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def get_col_ranges(data,col_name):
    col_data = data[col_name]
    min_value = col_data.min()
    max_value = col_data.max()
    values_count = col_data.nunique()
    values_range = col_data.unique()
    values_range.sort()
    values_range = values_range.tolist()
    return min_value,max_value,values_count,values_range

In [3]:
def value_range(column_names,df): # Getting the feature statistics
    min_value_list,max_value_list,values_count_list,values_range_list = [],[],[],[]
    for i in column_names:
        min_value,max_value,values_count,values_range = get_col_ranges(df,i)
        min_value_list.append(min_value)
        max_value_list.append(max_value)
        values_count_list.append(values_count)
        values_range_list.append(values_range)
    return min_value_list,max_value_list,values_count_list,values_range_list

In [4]:
def Disturbance_range(column_names,min_value_list,max_value_list,values_count_list):
    x_list = []
    for j in range(0,len(column_names)):
        if min_value_list[j]==0 and max_value_list[j]==1 and values_count_list[j]!=2:
            x_list.append ([i / 10 for i in range(11)])
        elif (max_value_list[j]-min_value_list[j])/values_count_list[j] > 100:
            num_of_random_values = 30 
            for _ in range(num_of_random_values):
                random_data = random.randint(min(values_range_list[j]), max(values_range_list[j]))
                values_range_list[j].append(random_data)
                values_range_list[j].sort()
            x_list.append(values_range_list[j])
        else:
            x_list.append ([i for i in range(int(min_value_list[j]),int(max_value_list[j])+1)])
    return x_list

In [5]:
def Fresh_breeze(x,column_names,min_value_list,max_value_list,values_count_list,init_val):
    y_list = [[] for _ in range(len(column_names))]
    interval = [i / 10 for i in range(11)]
    rel_out = np.argmax(init_val,axis=1) 
    start = time.perf_counter()
    for j in range(len(column_names)):
        if min_value_list[j] == 0 and max_value_list[j] == 1 and values_count_list[j]!=2:
            values = [interval[i] for i in range(11)]
        elif (max_value_list[j]-min_value_list[j])/values_count_list[j] > 100:
            values = [x_list[j][i] for i in range(len(x_list[j]))]
        elif len(x_list[j]) > 10000:
            num_partitions = 1000
            step = (max_value_list[j]-min_value_list[j])/num_partitions
            values = [start + k * step for k in range(num_partitions)]
            x_list[j] = values
        else:
            values = [i for i in range(int(min_value_list[j]), int(max_value_list[j]) + 1)]
        for i in values:
            modified_x = list(x.numpy())
            modified_x[0][j] = i
            modified_x = tf.convert_to_tensor(modified_x, dtype=tf.float32)
            rel = model.predict(modified_x)
            variety = rel - init_val      
            first_value = variety[0][rel_out]    
            y_list[j].append(first_value)
    end = time.perf_counter()
    print('Running time: %s Seconds'%(end-start))
    return y_list

In [6]:
def visualization(column_names,y_list):
    y_list_max = [max(sub_list) for sub_list in y_list]
    y_list_min = [min(sub_list) for sub_list in y_list]
    y_list_abs = [[abs(x) for x in inner_list] for inner_list in y_list]
    max_values = [max(sublist) for sublist in y_list_abs]
    for i in range(len(column_names)):
        plt.figure(figsize=(10,80))
        plt.subplot(len(column_names), 1, i+1)
        plt.plot(x_list[i], y_list[i],linestyle='-', color='r')
        max_index = y_list[i].index(max(y_list[i]))
        min_index = y_list[i].index(min(y_list[i]))
        x_max, y_max = x_list[i][max_index], y_list[i][max_index]
        x_min, y_min = x_list[i][min_index], y_list[i][min_index]
        plt.annotate(f'Max: ({x_max}, {y_max})', (x_max, y_max), textcoords="offset points", xytext=(0,10), ha='center', color='red')
        plt.annotate(f'Min: ({x_min}, {y_min})', (x_min, y_min), textcoords="offset points", xytext=(0,10), ha='center', color='green')
        plt.xlabel(column_names[i])
        plt.ylabel('Forecast change')
        plt.ylim(y_list_min[i]*1.1,y_list_max[i]*1.5)
        plt.grid(True)
    return max_values

In [7]:
def feature_important(column_names,threshold,max_values):
    total_sum = sum(max_values)
    percentages = [(value / total_sum) * 100 for value in max_values]
    merged_values = []
    merged_categories = []
    other_total = 0
    for category, value in zip(column_names, max_values):
        if value / total_sum * 100 < threshold:
            other_total += value
        else:
            merged_values.append(value)
            merged_categories.append(category)
    merged_values.append(other_total)
    merged_categories.append("others")
    merged_values = np.concatenate(merged_values)
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].plot(merged_categories, merged_values, marker='o', color='#813C85', label='line')
    axs[0].bar(merged_categories, merged_values, alpha=0.4, color='#DE3F7C', label='bar')
    axs[0].tick_params(axis='x', rotation=70)
    axs[1].pie(merged_values, labels=merged_categories, autopct='%1.1f%%') 
    plt.show()